<a href="https://colab.research.google.com/github/FranciscoRamirezArias/Pruebas_Software_Aseguramiento_Calidad/blob/main/ReservationSystem_v0_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instituto Tecnologico de Monterrey**
# **Maestría en Inteligencia Artificial Aplicada**
# **Curso: Pruebas de Software y aseguramiento de la Calidad**
# **Clave: TC4017.10**
## Profesor Títular: Dr. Gerardo Padilla Zárate
## Profesora Asistente: Mtra. Viridiana Rodríguez González
## Estudiante: Francisco Javier Ramírez Arias
## Matrícula: A01316379

##Actividad: Actividad 6.2
##Ejercicio de Programación #3
##Descripción: Sistema de Resevación.

### **Creación de Directorio**

In [11]:
!mkdir hotel_project

mkdir: cannot create directory ‘hotel_project’: File exists


### **Instalación de Pylint**

In [37]:
pip install pylint

### **Instalación de Flake**

In [38]:
pip install flake8

---
---

### **Instalación de Coverage**

In [57]:
pip install coverage

---
---

### **Creación de las Clases**
Se definen las clases y la lógica básica dentro de los sistema de reservación.
En el bloque de código se crean 3 clases, que son:

1. **Hotel:** representa los hoteles en el sistema, y nos permite guardar datos de Identificación del Hotel, Nombre del Hotel, y Ubicación. Además con los metodos de reserve_room y cancel_room nos permiten controlar cuantas habitaciones quedan disponibles. El métod de display_info nos muestra un resumen de los datos del hotel.

2. **Customers:** nos permite almacenar los datos del cliente, entre los que se encuentran ID, nombre, email, y teléfono. Con el método update_info podemos cambiar los datos del cliente.

3. **Reservation:** esta clase nos permite enlazar las otras dos clases, conecta a un cliente con un hotel con un identificador de reserva único.

In [39]:
%%writefile hotel_project/models.py
"""
Clases para el Sistema de Reservación
El módulo define las entidades principales: Hotel, Customer, Reservation.
"""
# Importamos herramientas para crear clases y convertirlas a diccionarios
# Dict nos avisa que algunas funciones devuelven diccionarios
from dataclasses import dataclass, asdict
from typing import Dict


# Representa la clases Hotel
@dataclass
class Hotel:                 # Nombre de la clase
    """Representa la entidad Hotel con sus atributos y métodos."""
    hotel_id: str            # Atributo: Identificador del hotel
    name: str                # Atributo: Nombre del hotel
    location: str            # Atributo: Ubicación del hotel
    total_rooms: int         # Atributo: Habitaciones totales
    available_rooms: int     # Atributo: Habitaciones disponibles

# Método para reservar habitación
    def reserve_room(self) -> bool:                    # Método para reservar habitación
        """Decrementa el número de habitaciones"""
        if self.available_rooms > 0:                   # Verdadero reserva habitación
            self.available_rooms -= 1                  # Falso, no hay habitaciones
            return True
        return False

# Método para ver informacion de hotel
    def display_info(self) -> str:
        """Regresa un resumen de la información del hotel."""
        return (
        f"\nHotel ID: {self.hotel_id}\n"               # Regresa Hotel ID
        f"Name: {self.name}\n"                         # Regresa Nombre del hotel
        f"Location: {self.location}\n"                 # Regresa ubicación del hotel
        f"Rooms: {self.available_rooms}/"
        f"{self.total_rooms}\n"  # Regresa disponibilidad de habitaciones
    )

# Método para cancelar habitación
    def cancel_room(self) -> None:
        """Incrementa o cancela el número de habitaciones"""
        if self.available_rooms < self.total_rooms:
            self.available_rooms += 1

# Método para actualizar información de hotel
    def update_info(self, name=None, location=None,
                    total_rooms=None) -> None:
        """Modifica la información del hotel"""
        if name:
            self.name = name                    # Nombre
        if location:
            self.location = location            # Ubicación
        if total_rooms:
            self.total_rooms = total_rooms      # Número de habitaciones

    # La clase se convierte a diccionario
    # La que contiene los atributos de hotel que son:
    # Hotel_Id, Nombre, Ubicación, Total de Habitaciones, Habitaciones Disponibles
    def to_dict(self) -> Dict:
        """Convierte el objeto Hotel a un diccionario."""
        return asdict(self)


# Representa la clases Cliente
@dataclass
class Customer:                # Nombre de la clase
    """Representa la entidad Customer con sus atributos y métodos."""
    customer_id: str           # Atributo: Identificador del cliente
    name: str                  # Atributo: Nombre del cliente
    email: str                 # Atributo: Email del cliente
    phone: str                 # Atributo: Teléfono del cliente

# Método para actualizar información de cliente
    def update_info(self, name=None, email=None,
                    phone=None) -> None:
        """Modifica la información del cliente"""
        if name:
            self.name = name             # Nuevo nombre del cliente
        if email:
            self.email = email           # Nuevo email del cliente
        if phone:
            self.phone = phone           # Nuevo teléfono del cliente

    # La clase se convierte a diccionario y regresa
    # los atributos de cliente que son:
    # Identificador, nombre, email y telefono
    def to_dict(self) -> Dict:
        """Convierte el objeto Customer a un diccionario."""
        return asdict(self)


# Representa la clase de Reservación
@dataclass
class Reservation:             # Nombre de la clase
    """Representa la reservación"""
    reservation_id: str        # Atributo: Identificador de la reserva
    customer_id: str           # Atributo: Identificador del cliente
    hotel_id: str              # Atributo: Identificador del hotel

    # La clase se convierte a diccionario y regresa
    # los atributos de la reservacion que son:
    # Identificador de la reserva, cliente, y hotel
    def to_dict(self) -> Dict:
        """Convierte el objeto Reservation a un diccionario."""
        return asdict(self)

Overwriting hotel_project/models.py


---
---

### **Creación de las Instancias de Almacenamiento**
Este archivo se encarga de implementar el almacenamiento y lectura de los datos del sistema de reservación a través del uso de archivos JSON.

1. **load_data:** verifica si el archivo existe, intenta leer el contenido, y devuelve una lista de diccionarios.

2. **save_data():** recibe una lista de diccionarios y la guarda en formato JSON con indentación para mejor legibilidad, manejando posibles errores de lectura.

Módulo independiente, que separa la lógica de almacenamiento de los datos permite cargas y guardar datos.

In [40]:
%%writefile hotel_project/storage.py
"""
El módulo carga y almacena datos en archivo en formato JSON.
Incluye  manejo básico de errores y excepciones.
"""
import json
import os
from typing import List, Dict

# Definición de función de lectura
def load_data(filename: str) -> List[Dict]:
    """Carga los datos del archivo JSON """
    if not os.path.exists(filename):         # Verifica si el archivo existe
        return []                            # Regresa una lista vacía si no existe

    try:
        with open(filename, "r", encoding="utf-8") as file:  # Abre el archivo
            return json.load(file)                           # Carga los datos del archivo
    except (json.JSONDecodeError, IOError) as error:         #Verifica errores
        print(f"Error reading {filename}: {error}")          # Imprime el error
        return []

# Definición de función de escritura
def save_data(filename: str, data: List[Dict]) -> None:
    """Salva los datos en JSON."""
    try:
        with open(filename, "w", encoding="utf-8") as file:   # Escribe en el archivo
            json.dump(data, file, indent=4)                   # Guarda los datos en formato JSON
    except IOError as error:                                  # Verifica errores
        print(f"Error writing {filename}: {error}")           # Imprime el error

Overwriting hotel_project/storage.py




---
---



### **Creación de las Servicios**
Se definen las clases y la lógica básica dentro de los sistema de reservación.
En el bloque de código se crean 3 clases, que son:

1. **Hotel:** representa los hoteles en el sistema, y nos permite guardar datos de Identificación del Hotel, Nombre del Hotel, y Ubicación. Además con los metodos de reserve_room y cancel_room nos permiten controlar cuantas habitaciones quedan disponibles. El métod de display_info nos muestra un resumen de los datos del hotel.

2. **Customers:** nos permite almacenar los datos del cliente, entre los que se encuentran ID, nombre, email, y teléfono. Con el método update_info podemos cambiar los datos del cliente.

3. **Reservation:** esta clase nos permite enlazar las otras dos clases, conecta a un cliente con un hotel con un identificador de reserva único.

In [41]:
%%writefile hotel_project/services.py
"""
Service layer implementing persistent behaviors
"""

import uuid
from typing import Optional
from hotel_project.models import Hotel, Customer, Reservation
from hotel_project.storage import load_data, save_data


HOTELS_FILE = "hotels.json"
CUSTOMERS_FILE = "customers.json"
RESERVATIONS_FILE = "reservations.json"


class HotelService:

    @staticmethod
    def create_hotel(name: str, location: str,
                     total_rooms: int) -> Hotel:
        hotels = load_data(HOTELS_FILE)

        hotel = Hotel(
            hotel_id=str(uuid.uuid4()),
            name=name,
            location=location,
            total_rooms=total_rooms,
            available_rooms=total_rooms
        )

        hotels.append(hotel.to_dict())
        save_data(HOTELS_FILE, hotels)
        return hotel

    @staticmethod
    def delete_hotel(hotel_id: str) -> bool:
        hotels = load_data(HOTELS_FILE)
        updated = [h for h in hotels if h["hotel_id"] != hotel_id]

        if len(updated) == len(hotels):
            return False

        save_data(HOTELS_FILE, updated)
        return True

    @staticmethod
    def display_hotel(hotel_id: str) -> Optional[Hotel]:
        hotels = load_data(HOTELS_FILE)
        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                return Hotel(**hotel)
        return None

    @staticmethod
    def modify_hotel(hotel_id: str, **kwargs) -> bool:
        hotels = load_data(HOTELS_FILE)

        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                obj = Hotel(**hotel)
                obj.update_info(**kwargs)
                hotel.update(obj.to_dict())
                save_data(HOTELS_FILE, hotels)
                return True

        return False


class CustomerService:

    @staticmethod
    def create_customer(name: str, email: str,
                        phone: str) -> Customer:
        customers = load_data(CUSTOMERS_FILE)

        customer = Customer(
            customer_id=str(uuid.uuid4()),
            name=name,
            email=email,
            phone=phone
        )

        customers.append(customer.to_dict())
        save_data(CUSTOMERS_FILE, customers)
        return customer

    @staticmethod
    def delete_customer(customer_id: str) -> bool:
        customers = load_data(CUSTOMERS_FILE)
        updated = [c for c in customers
                   if c["customer_id"] != customer_id]

        if len(updated) == len(customers):
            return False

        save_data(CUSTOMERS_FILE, updated)
        return True

    @staticmethod
    def display_customer(customer_id: str) -> Optional[Customer]:
        customers = load_data(CUSTOMERS_FILE)
        for customer in customers:
            if customer["customer_id"] == customer_id:
                return Customer(**customer)
        return None

    @staticmethod
    def modify_customer(customer_id: str, **kwargs) -> bool:
        customers = load_data(CUSTOMERS_FILE)

        for customer in customers:
            if customer["customer_id"] == customer_id:
                obj = Customer(**customer)
                obj.update_info(**kwargs)
                customer.update(obj.to_dict())
                save_data(CUSTOMERS_FILE, customers)
                return True

        return False


class ReservationService:

    @staticmethod
    def create_reservation(customer_id: str,
                           hotel_id: str) -> Optional[Reservation]:

        hotels = load_data(HOTELS_FILE)
        reservations = load_data(RESERVATIONS_FILE)

        for hotel in hotels:
            if hotel["hotel_id"] == hotel_id:
                obj = Hotel(**hotel)

                if not obj.reserve_room():
                    return None

                hotel.update(obj.to_dict())
                save_data(HOTELS_FILE, hotels)

                reservation = Reservation(
                    reservation_id=str(uuid.uuid4()),
                    customer_id=customer_id,
                    hotel_id=hotel_id
                )

                reservations.append(reservation.to_dict())
                save_data(RESERVATIONS_FILE, reservations)
                return reservation

        return None

    @staticmethod
    def cancel_reservation(reservation_id: str) -> bool:
        reservations = load_data(RESERVATIONS_FILE)
        hotels = load_data(HOTELS_FILE)

        for reservation in reservations:
            if reservation["reservation_id"] == reservation_id:

                hotel_id = reservation["hotel_id"]

                reservations = [
                    r for r in reservations
                    if r["reservation_id"] != reservation_id
                ]

                for hotel in hotels:
                    if hotel["hotel_id"] == hotel_id:
                        obj = Hotel(**hotel)
                        obj.cancel_room()
                        hotel.update(obj.to_dict())

                save_data(RESERVATIONS_FILE, reservations)
                save_data(HOTELS_FILE, hotels)
                return True

        return False

Writing hotel_project/services.py


---
---

### **Creación del Código Principal**
Este archivo implementa la interfaz de usuario por consola del sistema de reservación. La función principal de este módulo es controlar la interacción con el usuario y controlar la lógica del programa, asi como el control del flujo.

El programa esta dividiso en:


1.   Menú principal
2.   Submenús:
* Gestión de hoteles
* Gestión de clientes
* Gestión de reservaciones
3.   Bucle principal

In [42]:
%%writefile hotel_project/main.py
"""
Programa principal para el sistema de reservación
Este módulo implementa la interfaz de usuario por consola
y controla la lógica del programa.
"""

from hotel_project.services import (
    HotelService,
    CustomerService,
    ReservationService
)

# Definicion de función menu principal
def main_menu():
    print("\n==== HOTEL MANAGEMENT SYSTEM ====")    # Imprime encabezado
    print("1. Hotel Management")                    # Imprime opciones
    print("2. Customer Management")                 # Imprime opciones
    print("3. Reservation Management")              # Imprime opciones
    print("4. Exit")


# ---------------- MENU DEL HOTEL ---------------- #

# Definicion de función menu de Hotel
def hotel_menu():
    print("\n---- HOTEL MANAGEMENT ----")          # Imprime encabezado
    print("1. Create Hotel")                       # Imprime opciones
    print("2. Delete Hotel")                       # Imprime opciones
    print("3. Display Hotel Information")          # Imprime opciones
    print("4. Modify Hotel")                       # Imprime opciones
    print("5. Back")                               # Imprime opciones


# Definicion de función para manejar el menu de Hotel
def handle_hotel():
    # Ciclo infinito para mantener el menu de hotel activo
    while True:
        hotel_menu()
        option = input("Select an option: ")         # Captura la seleccion de usuario

        try:
            if option == "1":                        # Crea un hotel
                name = input("Hotel name: ")         # Nombre del hotel
                location = input("Location: ")       # Ubicación del hotel
                total_rooms = int(input("Total rooms: "))    # Total de habitaciones
                # Delega la creación a la capa de servicio
                hotel = HotelService.create_hotel(
                    name,
                    location,
                    total_rooms
                )
                print(f"Hotel created with ID: {hotel.hotel_id}") # Imprime confirmación

            elif option == "2":                      # Elimina un hotel
                hotel_id = input("Hotel ID: ")       # ID de Identificación
                result = HotelService.delete_hotel(hotel_id)  # Borrado del hotel
                print("Hotel deleted." if result else "Hotel not found.") #Informa la operación

            elif option == "3":                      # Información del hotel
                hotel_id = input("Hotel ID: ")       # ID de Identificación
                hotel = HotelService.display_hotel(hotel_id)   # Pasa a la capa de servicio

                if hotel:                            # Valida la existencia
                    print(hotel.display_info())
                else:
                    print("Hotel not found.")

            elif option == "4":                      # Modifica un hotel
                hotel_id = input("Hotel ID: ")       # ID de Identificacion
                name = input("New name (leave blank to skip): ")          #Nuevo nombre
                location = input("New location (leave blank to skip): ")  #Nueva ubicacion
                total_rooms_input = input(
                    "New total rooms (leave blank to skip): "             #Nuevo total
                )
# Prepara la actualización del diccionario
                kwargs = {}
                if name:
                    kwargs["name"] = name
                if location:
                    kwargs["location"] = location
                if total_rooms_input:
                    kwargs["total_rooms"] = int(total_rooms_input)  #Conversión
# Delega la actualizacion a la capa de servicio
                result = HotelService.modify_hotel(
                    hotel_id,
                    **kwargs
                )
                print("Hotel updated." if result else "Hotel not found.")
# Salida del menu principal
            elif option == "5":
                break
# Opción invalida
            else:
                print("Invalid option.")
# Maneja las conversiones numericas
        except ValueError:
            print("Invalid numeric input.")
        except Exception as error:
            print(f"Unexpected error: {error}")


# ---------------- MENU DEL CLIENTE ---------------- #
# Definicion de función menu de cliente
def customer_menu():
    print("\n---- CUSTOMER MANAGEMENT ----")         # Imprime el encabezado
    print("1. Create Customer")                      # Imprime opciones
    print("2. Delete Customer")                      # Imprime opciones
    print("3. Display Customer Information")         # Imprime opciones
    print("4. Modify Customer")                      # Imprime opciones
    print("5. Back")                                 # Imprime opciones

# Definicion de función manejar el menu del cliente
def handle_customer():
    while True:
        customer_menu()
        option = input("Select an option: ")     # Captura la seleccion del usuario

        try:
            if option == "1":                    # Crea un cliente
                name = input("Customer name: ")  # Nombre del cliente
                email = input("Email: ")         # correo electronico
                phone = input("Phone: ")         # telefono
# Delega la creacion del cliente a la capa de servicio
                customer = CustomerService.create_customer(
                    name,
                    email,
                    phone
                )
                # Despliega un mensaje de confirmación
                print(f"Customer created with ID: {customer.customer_id}")

            elif option == "2":                                        # Elimina un cliente
                customer_id = input("Customer ID: ")                   # ID Cliente
                result = CustomerService.delete_customer(customer_id)
                print("Customer deleted."
                      if result else "Customer not found.")            # Informa la operación

            elif option == "3":                                        # Informacion del cliente
                customer_id = input("Customer ID: ")                   # ID Cliente
                customer = CustomerService.display_customer(
                    customer_id
                )                                                 # Pasa a la capa de servicio

                if customer:                                      # Valida la existencia
                    print(customer)
                else:                                             # Informa que no existe
                    print("Customer not found.")

            elif option == "4":                                        # Modifica un cliente
                customer_id = input("Customer ID: ")                   # ID Cliente
                name = input("New name (leave blank to skip): ")       # Nuevo nombre
                email = input("New email (leave blank to skip): ")     # Nuevo correo
                phone = input("New phone (leave blank to skip): ")     # Nuevo telefono

                kwargs = {}                                           # Prepara la actualización del diccionario
                if name:
                    kwargs["name"] = name                             #Se incluye en el diccionario
                if email:
                    kwargs["email"] = email                           #Se incluye en el diccionario
                if phone:
                    kwargs["phone"] = phone                           #Se incluye en el diccionario
# Delega la actualizacion a la capa de servicio
                result = CustomerService.modify_customer(
                    customer_id,
                    **kwargs
                )
                print("Customer updated."
                      if result else "Customer not found.")         # Imprime la actualizacion

            elif option == "5":                                     # Salir del menu
                break

            else:
                print("Invalid option.")                           # Opcion invalida

        except Exception as error:                                 # Maneja errores
            print(f"Unexpected error: {error}")


# ---------------- MENU DE RESERVACION ---------------- #
# Definicion de función del menu de reservación
def reservation_menu():
    print("\n---- RESERVATION MANAGEMENT ----")
    print("1. Create Reservation")
    print("2. Cancel Reservation")
    print("3. Back")

# Definicion de función para manejar reservación
def handle_reservation():
    while True:
        reservation_menu()
        option = input("Select an option: ")             # Captura la selección del usuario

        try:
            if option == "1":                            # Crea una reservación
                customer_id = input("Customer ID: ")     # ID Cliente
                hotel_id = input("Hotel ID: ")           # ID Hotel

                reservation = (
                    ReservationService.create_reservation(
                        customer_id,
                        hotel_id
                    )                                    # Pasa la reserva a la capa de servicios
                )

                if reservation:
                    print(
                        f"Reservation created with ID: "
                        f"{reservation.reservation_id}"
                    )                                    # Imprime confirmación
                else:
                    print("Reservation failed.")

            elif option == "2":                                  # Cancela una reservación
                reservation_id = input("Reservation ID: ")       # Solicita ID
                result = ReservationService.cancel_reservation(
                    reservation_id
                )                                                # Pasa a la capa de servicios
                print("Reservation cancelled."
                      if result else "Reservation not found.")   # Impresiones

            elif option == "3":                                  # Salir del menu
                break

            else:
                print("Invalid option.")

        except Exception as error:
            print(f"Unexpected error: {error}")


# ---------------- CICLO PRINCIPAL ---------------- #

def main():
# Ciclo infinito que mantiene el sistema funcionando
    while True:
        main_menu()                             # Despliega el menu de navegacion
        option = input("Select an option: ")    # Captura la seleccion del usuario

        if option == "1":                       # Si la opcion es 1
            handle_hotel()                      # Redirecciona al modulo de Hotel
        elif option == "2":                     # Si la opcion es 2
            handle_customer()                   # Redirecciona al modulo de Cliente
        elif option == "3":                     # Si la opcion es 3
            handle_reservation()                # Redirecciona al modulo de Reservacion
        elif option == "4":                     # Si la opcion en 4
            print("Exiting system.")            # Sale del sistema
            break
        else:
            print("Invalid option.")            # Opcion invalida

# Asegura que el programa corra solo cuando se ejecuta directamente
# no cuando se importa como modulo
if __name__ == "__main__":
    main()

Overwriting hotel_project/main.py


---
---

### **Análisis del Código**

In [43]:
!pylint hotel_project/models.py hotel_project/services.py hotel_project/storage.py hotel_project/main.py
!flake8 hotel_project/models.py hotel_project/services.py hotel_project/storage.py hotel_project/main.py

************* Module services
hotel_project/services.py:7:0: E0401: Unable to import 'hotel_project.models' (import-error)
hotel_project/services.py:8:0: E0401: Unable to import 'hotel_project.storage' (import-error)
hotel_project/services.py:16:0: C0115: Missing class docstring (missing-class-docstring)
hotel_project/services.py:19:4: C0116: Missing function or method docstring (missing-function-docstring)
hotel_project/services.py:36:4: C0116: Missing function or method docstring (missing-function-docstring)
hotel_project/services.py:47:4: C0116: Missing function or method docstring (missing-function-docstring)
hotel_project/services.py:55:4: C0116: Missing function or method docstring (missing-function-docstring)
hotel_project/services.py:69:0: C0115: Missing class docstring (missing-class-docstring)
hotel_project/services.py:72:4: C0116: Missing function or method docstring (missing-function-docstring)
hotel_project/services.py:88:4: C0116: Missing function or method docstring (mis

---
---

### **Pruebas del Sistema**
El programa test_hotel_system implementa una serie de pruebas unitarias a traves del uso del framework unittest para validar el correcto funcionamiento del sistema de reservación. Las pruebas verifican:


*   Creación, modificación, eliminación y visualización de hoteles.
*   Creación, modificación, eliminación y visualización de clientes.
*   Creación y cancelación de reservaciones.
*   Manejo de casos negativos (IDs invalidos, hoteles sin habitacioens disponibles)
*   Manejo de errores en almacenamiento JSON

A grandes rasgos el programa de prueba valida la lógica de funcionamiento del programa, los datos, y el manejo de errores del sistema, se busca cumplir con una cobeetura del 85%.



In [50]:
%%writefile hotel_project/test_hotel_system.py
"""
Unidad de pruebas para el sistema de reservación
Este módulo implementa pruebas unitarias para el sistema de reservación,
valida la funcionalidad y el correcto funcionamiento del programa.+, incluye:
- Hoteles
- Clientes
- Reservaciones

La prueba utiliza el framework de pruebas unittest.
y verifica la logica, almacenamiento y correcto funcionamiento del programa.
"""

# Se importan librerias de unittest
import unittest
import os
import json
# Se importan las clases responsables de la lógica del sistema
# Estas clases se prueban
from hotel_project.services import (
    HotelService,                   # Maneja operaciones relacionadas con hotel
    CustomerService,                # Maneja operaciones relacionadas con clientes
    ReservationService              # Maneja operaciones relacionadas con reservaciones
)
from hotel_project.storage import load_data   # Carga datos JSON durante validacion


HOTELS_FILE = "hotels.json"                   # Archivo que almacena hoteles
CUSTOMERS_FILE = "customers.json"             # Archivo que almacena clientes
RESERVATIONS_FILE = "reservations.json"       # Archivo que almacena reservaciones


class TestHotelManagementSystem(unittest.TestCase):
    """
    La clase contiene las pruebas unitarias para el sistema de reservación,
    valida el correcto funcionamiento de hotel, cliente y reservacion,
    invluyendo los archivos JSON.
    """
    def setUp(self):
        """Resetea los archivos de prueba antes de cada prueba."""
        for file_name in [
            HOTELS_FILE,
            CUSTOMERS_FILE,
            RESERVATIONS_FILE
        ]:
            if os.path.exists(file_name):  # Si el archivo existe lo borra
                os.remove(file_name)       # para asegurar un ambiente limpio

    # ---------------- PRUEBAS DE HOTELES ---------------- #

# Prueba que verifica la creación de un hotel
    def test_create_hotel(self):                   # Prueba que verifica la creacion de un
        hotel = HotelService.create_hotel(         # de un hotel
            "Test Hotel", "NY", 5
        )
        self.assertEqual(hotel.total_rooms, 5)
        self.assertEqual(hotel.available_rooms, 5)
# Prueba que verifica el borrado de un hotel
    def test_delete_hotel_success(self):
        hotel = HotelService.create_hotel(
            "Delete Me", "LA", 3
        )
        result = HotelService.delete_hotel(hotel.hotel_id)
        self.assertTrue(result)
# Prueba que verifica el borrado de un hotel que esta en los datos
    def test_delete_hotel_not_found(self):
        result = HotelService.delete_hotel("invalid-id")
        self.assertFalse(result)
# Prueba que modifica la informacion de un hotel
    def test_modify_hotel(self):
        hotel = HotelService.create_hotel(
            "Old Name", "TX", 4
        )
        result = HotelService.modify_hotel(
            hotel.hotel_id,
            name="New Name"
        )
        self.assertTrue(result)
# Prueba que muestra la informacion de un hotel
    def test_display_hotel(self):
        hotel = HotelService.create_hotel(
            "Display Hotel", "FL", 6
        )
        found = HotelService.display_hotel(
            hotel.hotel_id
        )
        self.assertIsNotNone(found)

    # ---------------- PRUEBAS DE CLIENTES ---------------- #

# Prueba que crea un cliente
    def test_create_customer(self):
        customer = CustomerService.create_customer(
            "John", "john@mail.com", "123"
        )
        self.assertEqual(customer.name, "John")
#Prueba que borra un cliente
    def test_delete_customer_success(self):
        customer = CustomerService.create_customer(
            "Delete", "a@mail.com", "111"
        )
        result = CustomerService.delete_customer(
            customer.customer_id
        )
        self.assertTrue(result)
# Prueba que borra un cliente que no se encuentra
    def test_delete_customer_not_found(self):
        result = CustomerService.delete_customer(
            "invalid-id"
        )
        self.assertFalse(result)
# Prueba que modifica los datos de un cliente
    def test_modify_customer(self):
        customer = CustomerService.create_customer(
            "Old", "old@mail.com", "222"
        )
        result = CustomerService.modify_customer(
            customer.customer_id,
            name="Updated"
        )
        self.assertTrue(result)
# Prueba que muestra la informacion de un cliente
    def test_display_customer(self):
        customer = CustomerService.create_customer(
            "Jane", "jane@mail.com", "333"
        )
        found = CustomerService.display_customer(
            customer.customer_id
        )
        self.assertIsNotNone(found)

    # ---------------- PRUEBAS DE RESERVACIONES ---------------- #

# Prueba que crea una reservación
    def test_create_reservation_success(self):
        hotel = HotelService.create_hotel(
            "Res Hotel", "CA", 2
        )
        customer = CustomerService.create_customer(
            "Cust", "c@mail.com", "444"
        )

        reservation = ReservationService.create_reservation(
            customer.customer_id,
            hotel.hotel_id
        )

        self.assertIsNotNone(reservation)
# Prueba que crea una reservacion si no hay cuartos
    def test_create_reservation_no_rooms(self):
        hotel = HotelService.create_hotel(
            "Full Hotel", "NV", 1
        )
        customer = CustomerService.create_customer(
            "Cust", "c@mail.com", "555"
        )

        # First reservation succeeds
        ReservationService.create_reservation(
            customer.customer_id,
            hotel.hotel_id
        )

        # Second should fail
        reservation = ReservationService.create_reservation(
            customer.customer_id,
            hotel.hotel_id
        )

        self.assertIsNone(reservation)
# Prueba que cancela una reservacion
    def test_cancel_reservation_success(self):
        hotel = HotelService.create_hotel(
            "Cancel Hotel", "WA", 2
        )
        customer = CustomerService.create_customer(
            "Cust", "c@mail.com", "666"
        )

        reservation = ReservationService.create_reservation(
            customer.customer_id,
            hotel.hotel_id
        )

        result = ReservationService.cancel_reservation(
            reservation.reservation_id
        )

        self.assertTrue(result)
# Prueba que cancela una reservacion que no se encuentra
    def test_cancel_reservation_not_found(self):
        result = ReservationService.cancel_reservation(
            "invalid-id"
        )
        self.assertFalse(result)

    # ---------------- PRUEBAS DE ERROR DE ALMACENAMIENTO ---------------- #

# Prueba que simula un archivo JSON corrupto
    def test_invalid_json_handling(self):
        """Valida el contenido invalido del archivo"""
        with open(HOTELS_FILE, "w") as file:     # Escribe el contenido invalido
            file.write("INVALID JSON")

        data = load_data(HOTELS_FILE)            # Intenta cargar el archivo corrupto
        self.assertEqual(data, [])

# Punto de entrada de ejecucion de las pruebas.
if __name__ == "__main__":
    unittest.main()

Overwriting hotel_project/test_hotel_system.py


### **Ejecución de las Pruebas del Sistema**

In [54]:
import unittest
import sys
import os

# Add the directory containing 'hotel_project' to sys.path
# This makes 'hotel_project' discoverable as a top-level package
sys.path.insert(0, os.path.abspath('.'))

# Discover and run tests
loader = unittest.TestLoader()
suite = loader.discover('hotel_project') # Discover tests within 'hotel_project'
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_cancel_reservation_not_found (test_hotel_system.TestHotelManagementSystem.test_cancel_reservation_not_found) ... ok
test_cancel_reservation_success (test_hotel_system.TestHotelManagementSystem.test_cancel_reservation_success) ... ok
test_create_customer (test_hotel_system.TestHotelManagementSystem.test_create_customer) ... ok
test_create_hotel (test_hotel_system.TestHotelManagementSystem.test_create_hotel) ... ok
test_create_reservation_no_rooms (test_hotel_system.TestHotelManagementSystem.test_create_reservation_no_rooms) ... ok
test_create_reservation_success (test_hotel_system.TestHotelManagementSystem.test_create_reservation_success) ... ok
test_delete_customer_not_found (test_hotel_system.TestHotelManagementSystem.test_delete_customer_not_found) ... ok
test_delete_customer_success (test_hotel_system.TestHotelManagementSystem.test_delete_customer_success) ... ok
test_delete_hotel_not_found (test_hotel_system.TestHotelManagementSystem.test_delete_hotel_not_found) ... ok
test_de

Error reading hotels.json: Expecting value: line 1 column 1 (char 0)


<unittest.runner.TextTestResult run=15 errors=0 failures=0>

In [56]:
!coverage run -m unittest discover hotel_project
!coverage report -m

............Error reading hotels.json: Expecting value: line 1 column 1 (char 0)
...
----------------------------------------------------------------------
Ran 15 tests in 0.012s

OK
Name                                 Stmts   Miss  Cover   Missing
------------------------------------------------------------------
hotel_project/models.py                 50      5    90%   32, 53, 55, 81, 83
hotel_project/services.py              109      5    95%   52, 66, 105, 119, 151
hotel_project/storage.py                18      2    89%   28-29
hotel_project/test_hotel_system.py      77      1    99%   209
------------------------------------------------------------------
TOTAL                                  254     13    95%


---
---

### **Ejecución del Programa**

In [ ]:
!python -m hotel_project.main

### **Conclusiones**